In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 


%run "Custom Loss and Custom Metric.ipynb"


In [2]:
# Reading the data
train_df = pd.read_csv('../data/archive/train_df.csv')
# test_df = pd.read_csv('../data/archive/test_df.csv')
val_df = pd.read_csv('../data/archive/val_df.csv')

In [3]:
# Generate Ramdon sample for somputing statitics
datagen = tf.keras.preprocessing.image.ImageDataGenerator()
xcol = 'path'
ycol = list(train_df.columns[3:18])
random_sample_generator = datagen.flow_from_dataframe(
    train_df,
    x_col = 'path',
    y_col = ycol,
    target_size=(300, 300),
    batch_size=100,
    class_mode='raw',
    seed = 1000
)
random_sample_data = next(random_sample_generator)

Found 74833 validated image filenames.


In [4]:
# computing statistics for normalize layer 
normalize_layer = tf.keras.layers.experimental.preprocessing.Normalization()
normalize_layer.adapt(random_sample_data[0])
del random_sample_data
normalize_layer

In [5]:
BATCH_SIZE = 16
IMAGE_SIZE = (224,224)

In [6]:
# Creating data generators
datagen = tf.keras.preprocessing.image.ImageDataGenerator()

xcol = 'path'
ycol = list(train_df.columns[3:18])

train_generator = datagen.flow_from_dataframe(
    train_df,
    x_col = 'path',
    y_col = ycol,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='raw',
    seed = 100
)

# test_generator = datagen.flow_from_dataframe(
#     test_df,
#     x_col = 'path',
#     target_size=IMAGE_SIZE,
#     batch_size=1,
#     class_mode=None,
#     seed = 200
# )

val_generator = datagen.flow_from_dataframe(
    val_df,
    x_col = 'path',
    y_col = ycol,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='raw',
    seed = 300
)


Found 74833 validated image filenames.
Found 11504 validated image filenames.


In [7]:
# # Getting Sample data for display 
# sample_data = next(train_generator)

In [8]:
# # Preprocessing data
# rezize_data = tf.keras.layers.experimental.preprocessing.Resizing(IMAGE_SIZE[0],IMAGE_SIZE[1])(sample_data[0])
# rotated_data = tf.keras.layers.experimental.preprocessing.RandomRotation((-0.04,0.05))(rezize_data)
# rescale_data = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(rotated_data)
# normalized_data = normalize_layer(rotated_data)

In [9]:
# # Ploting the preprocessed data
# plt.figure(figsize=(20, 20))
# for i in range(BATCH_SIZE):
#     ax = plt.subplot(10, 4, i + 1)
#     plt.imshow(np.clip(rescale_data[i],0, 1))
#     plt.axis("off")

In [10]:
# # Ploting the unpreprocessed data
# plt.figure(figsize=(20, 20))
# for i in range(BATCH_SIZE):
#     ax = plt.subplot(8, 4, i + 1)
#     plt.imshow(sample_data[0][i].astype('uint8'))
#     plt.axis("off")

In [11]:
# calculating positive weights and negative weights
n_pos = train_df.iloc[:,3:18].sum(axis = 0).values
n_neg = train_df.shape[0] - n_pos
# wp = n_neg / train_df.shape[0]
# wn = n_pos / train_df.shape[0]
wp = (1 / n_pos) * (train_df.shape[0]) / 2
wn = (1 / n_neg) * (train_df.shape[0]) / 2
train_df.iloc[:,3:18].sum(axis = 0)

Cardiomegaly           1411
Emphysema              1337
Effusion               8161
No Finding            42358
Hernia                  102
Infiltration          12350
Mass                   3654
Nodule                 4192
Atelectasis            7606
Pneumothorax           2571
Pleural_Thickening     1990
Pneumonia               814
Fibrosis               1028
Edema                  1339
Consolidation          2680
dtype: int64

In [12]:
wp

array([ 26.51771793,  27.98541511,   4.58479353,   0.88333963,
       366.82843137,   3.02967611,  10.23987411,   8.92569179,
         4.91933999,  14.55328666,  18.80226131,  45.96621622,
        36.39737354,  27.94361464,  13.9613806 ])

In [13]:
wn

array([0.50960884, 0.50909573, 0.5612026 , 1.1521632 , 0.50068245,
       0.59882688, 0.52566768, 0.52967115, 0.55656953, 0.51778943,
       0.51365951, 0.50549859, 0.5069643 , 0.50910959, 0.51857165])

In [14]:
# Creating Model
def create_model(pretrained_model, wp, wn):
    inputs = tf.keras.Input(shape = (IMAGE_SIZE[0],IMAGE_SIZE[1],3))
    x = tf.keras.layers.experimental.preprocessing.Resizing(IMAGE_SIZE[0], IMAGE_SIZE[1])(inputs)
    x = tf.keras.layers.experimental.preprocessing.RandomRotation((-0.04,0.05))(x)
    x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(x)
    x = pretrained_model(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(128, activation = 'relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    outputs = tf.keras.layers.Dense(15, activation = 'sigmoid')(x)
    
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss= WeightedBinaryCrossEntropy(wp = wp, wn = wn),
              metrics=[tfa.metrics.F1Score(num_classes = 15, average = 'macro', threshold = 0.5,name = 'f1_macro'), MTCF1Score(15), tfa.metrics.F1Score(num_classes = 15, average = 'micro', threshold = 0.5,name = 'f1_micro')])
    
    return model


vgg16 = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape = (IMAGE_SIZE[0],IMAGE_SIZE[1],3))
# vgg16.trainable = False
model = create_model(vgg16, wp, wn)
# tf.keras.utils.plot_model(model, 'my_first_model.png', show_shapes=True)
# model.summary()


In [15]:
# Creating callbacks
weight_path="{}_weights_best.h5".format('VGG')

checkpoint = tf.keras.callbacks.ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min')

early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=3)

callbacks_list = [checkpoint, early]

In [16]:
# Training model
model.fit(
    train_generator, 
    epochs=10, 
    steps_per_epoch = np.ceil(train_generator.n/BATCH_SIZE), 
    validation_data = val_generator,
    validation_steps = np.ceil(val_generator.n/BATCH_SIZE),
    callbacks = callbacks_list
) 


Epoch 1/10
4678/4678 [==============================] - ETA: 0s - loss: 10.6454 - f1_macro: 0.1072 - mtc_f1_score: 0.0237 - f1_micro: 0.1176
Epoch 00001: val_loss improved from inf to 9.35231, saving model to VGG_weights_best.h5
4678/4678 [==============================] - 3226s 690ms/step - loss: 10.6454 - f1_macro: 0.1072 - mtc_f1_score: 0.0237 - f1_micro: 0.1176 - val_loss: 9.3523 - val_f1_macro: 0.0388 - val_mtc_f1_score: 0.0271 - val_f1_micro: 0.0513
Epoch 2/10
4678/4678 [==============================] - ETA: 0s - loss: 10.6543 - f1_macro: 0.1074 - mtc_f1_score: 0.0257 - f1_micro: 0.1136
Epoch 00002: val_loss improved from 9.35231 to 9.31544, saving model to VGG_weights_best.h5
4678/4678 [==============================] - 2811s 601ms/step - loss: 10.6543 - f1_macro: 0.1074 - mtc_f1_score: 0.0257 - f1_micro: 0.1136 - val_loss: 9.3154 - val_f1_macro: 0.0764 - val_mtc_f1_score: 0.0268 - val_f1_micro: 0.1427
Epoch 3/10
4678/4678 [==============================] - ETA: 0s - loss: 10.6

KeyboardInterrupt: 

In [ ]:
# Loading Model
saved_model = tf.keras.models.load_model('xray_class_weights_best.h5')
saved_model.evaluate(
    val_generator,
    steps = np.ceil(val_generator.n/BATCH_SIZE)
)

In [ ]:
# Make Predictions
y_preds = saved_model.predict(
    val_generator,
)

In [ ]:
y_true = val_df.iloc[:,3:].values

In [ ]:
from sklearn.metrics import roc_curve, auc
fig, c_ax = plt.subplots(1,1, figsize = (9, 9))
for (idx, c_label) in enumerate(ycol):
    fpr, tpr, thresholds = roc_curve(y_true[:,idx].astype(int), y_preds[:,idx])
    c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
c_ax.legend()
c_ax.set_xlabel('False Positive Rate')
c_ax.set_ylabel('True Positive Rate')
fig.savefig('barely_trained_net.png')

In [ ]:
# EPOCHS = 1
# optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
# loss_fn = tf.keras.losses.BinaryCrossentropy()

In [ ]:
# for epochs in range(EPOCHS):
#     for step, (X_batch_train, y_batch_train) in enumerate(train_generator):
#         with tf.GradientTape() as tape:
#             y_preds = model(X_batch_train, training = True)
#             loss_val = loss_fn(y_batch_train, y_preds)
#         gradients = tape.gradient(loss_val, model.trainable_weights)
#         optimizer.apply_gradients(zip(gradients, model.trainable_weights))
#         if step % 200 == 0:
#             t0
#         print("Training loss (for one batch) at step %d: %.4f"% (step, float(loss_val)), end = '\r')
#         if step == np.ceil(train_generator.n/BATCH_SIZE):
#             break
#     print()
    